# Basic usage of SFRmaker in a scripting context
This example illustrates basic usage of SFRmaker in a scripting context. For examples of using SFRmaker with a configuration file, see the MERAS and Tyler Forks examples.

In [1]:
import sys
sys.path.append('../python_packages_static')
import numpy as np
import matplotlib.pyplot as plt
import flopy
import geopandas
import sfrmaker


AttributeError: partially initialized module 'rasterio' has no attribute '_loading' (most likely due to a circular import)

### Input requirements
The most basic input requirements of SFRmaker are hydrography and a model grid. Optionally, a model and a DEM can be input, as demonstrated below. See the [documentation](https://aleaf.github.io/sfrmaker/inputs.html) for a more detailed description of inputs.

#### Hydrography
In this example, we will use data that has been downloaded from [NHDPlus](https://nhdplus.com/NHDPlus/NHDPlusV2_data.php). The original file structure in the download has been maintained, allowing us to simply supply SFRmaker with a path to the NHDPlus files:

In [2]:
NHDPlus_paths = '../tylerforks/NHDPlus/'

If we were dealing with more than one drainage basin, the NHDPlus file paths could be included in a list:

In [3]:
NHDPlus_paths_list = ['/NHDPlusGL/NHDPlus04/',
                      '/NHDPlusMS/NHDPlus07/']

### Creating a ``Lines`` instance from NHDPlus
The ``sfrmaker.Lines`` class includes functionality for reading and processing hydrography flowlines. This example shows how to create a ``Lines`` instance from NHDPlus data.

For large hydrography datasets, it is advantageous to filter the data when it is read in. The ``filter`` argument to ``sfrmaker.Lines`` accepts a shapefile path or tuple of bounding box coordinates. In either case, a bounding box tuple is created and passed to the ``filter`` method in the [``fiona`` package](https://fiona.readthedocs.io/en/latest/manual.html), which is fast.

In [4]:
lns = sfrmaker.Lines.from_nhdplus_v2(NHDPlus_paths='../tylerforks/NHDPlus/',
                            bbox_filter='../tylerforks/grid.shp')

NameError: name 'sfrmaker' is not defined

Alternatively, ``sfrmaker.Lines`` can be instantiated with separate arguments for each NHDPlus file used:

In [ ]:
lines = sfrmaker.Lines.from_nhdplus_v2(NHDFlowlines='../tylerforks/NHDPlus/NHDSnapshot/Hydrography/NHDFlowline.shp',
                                       PlusFlowlineVAA='../tylerforks/NHDPlus/NHDPlusAttributes/PlusFlowlineVAA.dbf',
                                       PlusFlow='../tylerforks/NHDPlus/NHDPlusAttributes/PlusFlow.dbf',
                                       elevslope='../tylerforks/NHDPlus/NHDPlusAttributes/elevslope.dbf',
                                       bbox_filter=(-90.625, 46.3788, -90.4634, 46.4586))

### Creating a ``Lines`` instance from custom hydrography
Alternatively, a ``Lines`` instance can be created from any hydrography that includes the pertinent attribute fields, which must be specified:

In [ ]:
custom_lines = sfrmaker.Lines.from_shapefile(shapefile='../meras/flowlines.shp',
                                             id_column='COMID',  # arguments to sfrmaker.Lines.from_shapefile
                                             routing_column='tocomid',
                                             width1_column='width1',
                                             width2_column='width2',
                                             up_elevation_column='elevupsmo',
                                             dn_elevation_column='elevdnsmo',
                                             name_column='GNIS_NAME',
                                             attr_length_units='feet',  # units of source data
                                             attr_height_units='feet',  # units of source data
                                             )

### Specifying a model grid from a flopy `StructuredGrid` instance
The next step is to specify a model grid. One option is to specify a flopy `StructuredGrid` instance. SFRmaker will then use this internally to create an instance of its own `StructuredGrid` class. In this case, the row and column spacing must be provided in the units of the projected coordinate reference system (CRS) that the model is in, which is typically meters. In this case, our model grid spacing is 250 feet, so we have to convert. 

Specifying a CRS (via the ``proj4`` argument) is also important, as it allows SFRmaker to automatically reproject any input data to the same CRS as the model grid. [The best way to do this is with an EPSG code](https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems), as shown below.

See the flopy documentation for more details about `StructuredGrid`.

In [ ]:
delr = np.array([250 * 0.3048] * 160)  # cell spacing along a row
delc = np.array([250 * 0.3048] * 111)  # cell spacing along a column

flopy_grid = flopy.discretization.StructuredGrid(delr=delr, delc=delc,
                                                 xoff=682688, yoff=5139052,  # lower left corner of model grid
                                                 angrot=0,  # grid is unrotated
                                                 # projected coordinate system of model (UTM NAD27 zone 15 North)
                                                 proj4='epsg:26715'
                                                 )

### Specifying a model grid from a shapefile
Another option is to create an SFRmaker `StructuredGrid` directly using a shapefile. While the basic underpinings are in place for SFRmaker to support unstructured grids, this option hasn't been fully implemented yet.

Attribute fields with row and column information must be specified. An polygon defining the area where the SFR network will be created can optionally be specified here, or later in the creation of ``SFRData`` (see below).

In [ ]:
grid = sfrmaker.StructuredGrid.from_shapefile(shapefile='../tylerforks/grid.shp',
                                              icol='i',  # attribute field with row information
                                              jcol='j',  # attribute field with column information
                                              active_area='../tylerforks/active_area.shp'
                                              )

With specification of ``active_area``, the grid created above has an **``isfr``** array attribute designating which cells can have SFR:

In [ ]:
plt.imshow(grid.isfr, interpolation='nearest')

### Using the ``modelgrid`` attached to a flopy model
If no grid is supplied as input, SFRmaker will try to use ``modelgrid`` attribute attached to a supplied flopy model instance. This only works if ``modelgrid`` is valid. Loading a flopy model with a valid model grid requires the grid information to be specified in the namefile header, and that the model be in the same units as the projected CRS. See the flopy documentation for more details

### Specifying a model
While a model is not required to run SFRmaker, specifying a model is advantageous in that it allows SFRmaker to assign valid model layers for reaches. Models are specified as flopy model instances, which can be loaded or created from scratch (see the flopy documentation). In this case, we are loading a model:

In [ ]:
m = flopy.modflow.Modflow.load('tf.nam', model_ws='../tylerforks/tylerforks')
m

### Creating an SFRData instance
The ``sfrmaker.SFRData`` class is the primary object for creating or modifying an SFR dataset. A ``SFRData`` instance can be created from the ``Lines`` class using the ``to_sfr()`` method.

Either a flopy ``StructuredGrid`` or sfrmaker ``StructuredGrid`` can be supplied. While MODFLOW and flopy support specification of length units, these aren't always specified in the model input files, so it is good practice to specifiy the units explicitly to SFRmaker.

In [ ]:
sfrdata = lines.to_sfr(grid=flopy_grid, model=m, model_length_units='feet')

### The ``SFRData`` class
Internally, the SFRData class mostly uses the data model for the MODFLOW-2005 style SFR2 package (which includes MODFLOW-NWT), which organizes the input into segments and reaches. Segment and reach data are stored in the ``reach_data`` and ``segment_data`` attribute DataFrames. In addition to the MODFLOW-2005 data, reach numbers and their routing connections (as needed for MODFLOW-6) are also stored. On writing of SFR package input, MODFLOW-6 style input can be created via the ``sfrmaker.mf5to6`` module.

In [ ]:
sfrdata.reach_data.loc[sfrdata.reach_data['rno'] == 1, 'strhc1'] = 5

In [ ]:
sfrdata.reach_data.head()

In [ ]:
sfrdata.segment_data.head()

### Sampling streambed top elevations from a DEM
While the above dataset contains streambed top elevations read from NHDPlus, a DEM can be sampled to obtain more accurate elevations. If the DEM elevation units are specified, SFRmaker will convert the elevations to model units if needed. See the Streambed_elevation_demo notebook for more information on how this works.

In [ ]:
sfrdata.set_streambed_top_elevations_from_dem('../tylerforks/dem_26715.tif', 
                                              elevation_units='meters')

### Assigning layers to the reaches
Once we have a valid set of streambed elevations, the reaches can be assigned to model layers.

In [ ]:
sfrdata.assign_layers()

In [ ]:
sfrdata.reach_data.head()

### Running diagnostics
`SFRData` includes a `run_diagnostics()` method that executes the Flopy checker on the SFR package input. While the Flopy checks are only implemented for MODFLOW-2005 style SFR packages, `run_diagnostics()` works on MODFLOW-6 packages as well, by working of an attached MODFLOW-2005 representation of the MODFLOW-6 SFR input. The following checks are executed:
* **NaNs** (not a number values, which will cause MODFLOW to crash)
* **consecutive segment and reach numbering**
* **segment numbering that only increases** in the downstream direction
* **circular routing sequences** (reaches or segments routing back to themselves)
* **routing connection proximity:** The length of routing connections, as determined by the distance between their cell centers, is compared to 1.25 times the diagonal distance across the cell containing the upstream reach. This benchmark length is equivalent to a diagonal connection between two square cells, where one cell had a spacing of 1.5 times the other cell (the maximum recommended increase in spacing between two adjacent cells; see Anderson and others, 2015). While this routing proximity check can be helpful for identifying egregious routing issues, in practice, it may not be uncommon for valid routing connections to violate this rule. For example, if the ``one_reach_per_cell`` option is used in constructing the SFR package with a large grid size, there may be numerous routing connections that extend across several cells. When non-adjacent routing connections are identified, the user can check the shapefile of routing connections to verify the results.
* **overlapping conductances:** This check identifies instances of collocated reaches, where more than one reach has a connection to the groundwater flow solution in that cell. Collocated reaches may or may not present an issue for a particular model solution, but can potentially promote instability or spurious circulation of flow between the collocated reaches.
* **spurious streambed top elevations:** Values < -10 or > 15,000 are flagged
* **downstream rises in streambed elevation**
* **inconsistencies between streambed elevation and the model grid:** This check looks for streambed bottoms below their respective cell bottoms, which will cause MODFLOW to halt execution, and streambed tops that are above the model top. The latter issue will not prevent MODFLOW from running, but may indicate problems with the input if the differences are large. For coarse grids in areas of high stream density (where there are often multiple streams in a cell), it may be impossible to ensure that all streambed top elevations are at or below the model top. Oftentimes, these reaches are dry anyways (for example, and ephemeral gully that runs off a bluff into a perennial stream, where the cell elevation is based on the elevation of the perennial stream). In any case, it is good practice to check the largest violations by comparing the SFRmaker shapefile output to a DEM and other data in a GIS environment.
* **suprious slopes:** slopes of less than 0.0001 (which can lead to excessive stages with the SFR package's Manning approximation) and greater than 1.0 are flagged.

In [ ]:
sfrdata.run_diagnostics(verbose=False)

### Writing an SFR package
Now we can write the package. By default, a MODFLOW-2005 style package is written to the current working directory or the model workspace of the attached model instance.

In [ ]:
sfrdata.write_package()

### Writing a MODFLOW-6 SFR package
Alternatively, a MODFLOW-6 style SFR package can be written by specifying ``version='mf6'``

In [ ]:
sfrdata.write_package(version='mf6')

### Writing tables
The ``reach_data`` and ``segment_data`` tables can be written out to csv files for further processing:

In [ ]:
sfrdata.write_tables()

and then read back in to create an ``SFRData`` instance

In [ ]:
sfrdata2 = sfrmaker.SFRData.from_tables('tables/tf_sfr_reach_data.csv', 'tables/tf_sfr_segment_data.csv',
                                        grid=grid)

### Writing shapefiles
Shapefiles can also be written to visualize the SFR package. These include:
* hydrography linestrings associated with each reach
* model cell polygons associated with each reach
* outlet locations where water is leaving the model
* routing connections (as lines drawn between cell centers)
* the locations of specified inflows or any other period data
* observation locations (if observations were supplied as input)

In [ ]:
sfrdata.write_shapefiles()